# Import Packages

In [2]:
import doubleml as dml
from doubleml.datasets import make_irm_data
import pandas as pd
import numpy as np
import lightgbm as lgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, average_precision_score, make_scorer, mean_absolute_error, median_absolute_error
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse.linalg import lobpcg
# import causalml.inference.meta as cml

# Load Data

In [3]:
df = pd.read_csv("../data/prepped_data.csv", low_memory=False, index_col=0).drop_duplicates()

df = df[df["first_data_year"] >= 2021]

In [4]:
nulls = [col for col, val in df.isnull().any().to_dict().items() if val == True]
print(nulls)

[]


# Setup Data

In [5]:
cols_to_drop = ["policy_nr_hashed", "last_data_year", "first_data_year", "first_datapoint_year", "last_datapoint_year"]
df = df[[col for col in df.columns.to_list() if (col not in cols_to_drop)]]

In [6]:
categorical_features = []
continuous_features = []
binary_features = []

# Define a threshold for the maximum number of unique values for a categorical column
max_unique_values_for_categorical = 10

# Iterate through each column to determine if it's categorical, continuous, or binary
for column in df.columns:
    unique_values = df[column].nunique()
    if unique_values == 2:
        # If exactly 2 unique values, treat column as binary
        binary_features.append(column)
    elif (df[column].dtype == 'object' or unique_values <= max_unique_values_for_categorical) and unique_values > 2:
        # If object type or up to the threshold of unique values (and more than 2), treat as categorical
        categorical_features.append(column)
    else:
        # Otherwise, treat as continuous
        continuous_features.append(column)

print(f'Binary Features: {binary_features}')
print(f'Categorical Features: {categorical_features}')
print(f'Continuous Features: {continuous_features}')

df = pd.get_dummies(df, columns=categorical_features)

Binary Features: ['churn', 'last_allrisk basis', 'last_allrisk compleet', 'last_allrisk royaal', 'last_wa-extra', 'fake_alarm', 'policyholder_change', 'n_last_vs_peak', 'lpa']
Categorical Features: ['count', 'control_group', 'last_brand', 'last_type', 'last_fuel_type', 'last_postcode', 'last_product', 'nr_cars', 'max_nr_coverages', 'last_nr_coverages']
Continuous Features: ['welcome_discount', 'first_premium', 'last_premium', 'first_split', 'last_split', 'last_customer_age', 'last_accident_free_years', 'last_car_value', 'last_age_car', 'last_weight', 'accident_years', 'last_change_premium_abs', 'last_change_premium_perc', 'last_vs_first_split', 'cum_change_premium_abs', 'cum_change_premium_perc']


# Setup Models

In [7]:
best_d = {'colsample_bytree': 0.5651430631040584, 'learning_rate': 0.05024033157100756, 'max_depth': 70.0, 'min_child_samples': 33.0, 'min_data_in_leaf': 5.0, 'min_split_gain': 0.0024206836721644767, 'n_estimators': 54.0, 'num_leaves': 185.0, 'reg_alpha': 0.19913197144824663, 'reg_lambda': 0.19906785062440704, 'subsample': 0.9121630873508754, 'subsample_freq': 26.0}

best_params_d = {
    'max_depth': int(best_d['max_depth']),
    'n_estimators': int(best_d['n_estimators']),
    'num_leaves': int(best_d['num_leaves']),
    'min_child_samples': int(best_d['min_child_samples']),
    'colsample_bytree': best_d['colsample_bytree'],
    'subsample': best_d['subsample'],
    'subsample_freq': int(best_d['subsample_freq']),
    'reg_alpha': best_d['reg_alpha'],
    'reg_lambda': best_d['reg_lambda'],
    'min_split_gain': best_d['min_split_gain'],
    'learning_rate': best_d['learning_rate'],
    'min_data_in_leaf': int(best_d['min_data_in_leaf']),
}

ml_d = lgb.LGBMClassifier(
    objective='binary',
    force_row_wise=True,
    verbosity=-1,
    # is_unbalance=True,
    **best_params_d
)

In [8]:
best_y = {'colsample_bytree': 0.2983935721861137, 'learning_rate': 0.04740706929909022, 'max_depth': 59.0, 'min_child_samples': 18.0, 'min_data_in_leaf': 13.0, 'min_split_gain': 0.3863623673164322, 'n_estimators': 74.0, 'num_leaves': 54.0, 'reg_alpha': 0.1198683978345154, 'reg_lambda': 0.18168767473399486, 'subsample': 0.9841777438197711, 'subsample_freq': 27.0}

best_params_y = {
    'max_depth': int(best_y['max_depth']),
    'n_estimators': int(best_y['n_estimators']),
    'num_leaves': int(best_y['num_leaves']),
    'min_child_samples': int(best_y['min_child_samples']),
    'colsample_bytree': best_y['colsample_bytree'],
    'subsample': best_y['subsample'],
    'subsample_freq': int(best_y['subsample_freq']),
    'reg_alpha': best_y['reg_alpha'],
    'reg_lambda': best_y['reg_lambda'],
    'min_split_gain': best_y['min_split_gain'],
    'learning_rate': best_y['learning_rate'],
    'min_data_in_leaf': int(best_y['min_data_in_leaf']),
}

ml_y = lgb.LGBMClassifier(
    objective='binary',
    force_row_wise=True,
    verbosity=-1,
    # is_unbalance=True,
    **best_params_y
)

# Setup DML

In [9]:
# Define custom bin edges
bin_edges = [0, 0.8, 0.9, 0.999999, 1]
labels = [3, 2, 1, 0]

# Transform into bins based on custom edges
df['welcome_discount'] = pd.cut(df['welcome_discount'], bins=bin_edges, labels=labels).astype("int")

In [10]:
display(df)

,welcome_discount,churn,first_premium,last_premium,first_split,last_split,last_customer_age,last_accident_free_years,last_car_value,last_age_car,...,max_nr_coverages_1,max_nr_coverages_2,max_nr_coverages_3,max_nr_coverages_4,max_nr_coverages_5,last_nr_coverages_1,last_nr_coverages_2,last_nr_coverages_3,last_nr_coverages_4,last_nr_coverages_5
policy_nr_hashed,,,,,,,,,,,,,,,,,,,,,
0WK709p,0,0,4178.496,4178.496,0.829849,0.829849,36,0,478004.8,29,...,False,True,False,False,False,False,True,False,False,False
0WK70kZ,0,0,4171.776,4171.776,0.829575,0.829575,47,16,236454.4,21,...,False,True,False,False,False,False,True,False,False,False
0WK72B5,0,0,3792.768,3792.768,0.812544,0.812544,61,0,266918.4,1,...,False,False,True,False,False,False,False,True,False,False
0WK72VZ,2,0,2342.592,2342.592,1.000000,1.000000,68,43,185875.2,9,...,True,False,False,False,False,True,False,False,False,False
0WK72gq,0,0,3810.240,3810.240,0.813404,0.813404,59,20,584584.0,18,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zPmyxXA,0,0,4284.672,4233.600,0.836888,0.832063,41,11,270614.4,16,...,False,False,True,False,False,False,False,True,False,False
zPmyyWO,0,1,8041.152,8215.200,0.917419,0.917832,69,18,644952.0,10,...,False,False,True,False,False,False,False,True,False,False
zPmyydl,0,0,3186.624,3968.832,1.000000,1.000000,51,6,276404.8,24,...,True,False,False,False,False,True,False,False,False,False


In [11]:
df_test = df.sample(frac = 0.3, random_state = 0)
df_train = df[~df.index.isin(df_test.index.to_list())]

In [12]:
obj_dml_data = dml.DoubleMLData(df, 'churn', 'welcome_discount')

dml_irm_obj = dml.DoubleMLIRM(obj_dml_data, ml_y, ml_d)

dml_irm_obj.fit()

ValueError: Incompatible data. To fit an IRM model with DML exactly one binary variable with values 0 and 1 needs to be specified as treatment variable.

In [ ]:
print(dml_irm_obj.summary)

                     coef   std err          t          P>|t|     2.5 %  \
welcome_discount  0.08668  0.003317  26.128553  1.727775e-150  0.080178   

                    97.5 %  
welcome_discount  0.093182  
